### 1. Prepare input and target data for model

In [2]:
import pandas as pd
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn import preprocessing

# load the dataset
def load_dataset(filename, column):
	# load the dataset as a pandas DataFrame
	df = read_csv(filename)
	# split into input (X) and output (y) variables & convert to numPy array
	X = df.drop(column, axis = 1).values
	y = df[column].values
	return X, y

# prepare input data
def prepare_inputs(X_train, X_test):
	oe = OrdinalEncoder()
	oe.fit(X_train)
	X_train_enc = oe.transform(X_train)
	X_test_enc = oe.transform(X_test)
		
	# scale dataset
	scaler = preprocessing.MinMaxScaler()
	X_train_rescaled = scaler.fit_transform(X_train_enc)
	X_test_rescaled = scaler.fit_transform(X_test_enc)
	return X_train_rescaled, X_test_rescaled

# prepare target
def prepare_targets(y_train, y_test):
	le = LabelEncoder()
	le.fit(y_train)
	y_train_enc = le.transform(y_train)
	y_test_enc = le.transform(y_test)
	return y_train_enc, y_test_enc

# load the dataset
X, y = load_dataset('mushrooms.csv', 'class')
# split into train and test sets; 80/20 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# prepare input data
X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)
# prepare output data
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)

### 2. Compute Logistic Regression and accuracy

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from matplotlib import pyplot

# define how many features to select
number_of_features = 10
print('number of selected features: ', number_of_features, '\n')

# Feature reduction function
def select_features(X_train, y_train, X_test, reduction_func):
 fs = SelectKBest(score_func=reduction_func, k=number_of_features)
 fs.fit(X_train, y_train)
 X_train_fs = fs.transform(X_train)
 X_test_fs = fs.transform(X_test)
 return X_train_fs, X_test_fs

#### FEATURE SELECTION ####

# chi2 feature selection
X_train_fs_chi2, X_test_fs_chi2 = select_features(X_train_enc, y_train_enc, X_test_enc, chi2)

# mutual information feature selection
X_train_fs_mi, X_test_fs_mi = select_features(X_train_enc, y_train_enc, X_test_enc, mutual_info_classif)

#### MODEL FITTING ####

model = LogisticRegression(solver='lbfgs')

# all features
model.fit(X_train_enc, y_train_enc)
yhat = model.predict(X_test_enc)
accuracy = accuracy_score(y_test_enc, yhat)
print('Accuracy using ALL features: %.2f' % (accuracy*100))

# Chi2 features
model.fit(X_train_fs_chi2, y_train_enc)
yhat = model.predict(X_test_fs_chi2)
accuracy = accuracy_score(y_test_enc, yhat)
print('Accuracy using Chi2: %.2f' % (accuracy*100))

# Mutual Information featuresW
model.fit(X_train_fs_mi, y_train_enc)
yhat = model.predict(X_test_fs_mi)
accuracy = accuracy_score(y_test_enc, yhat)
print('Accuracy using Mutual Information: %.2f' % (accuracy*100))

number of selected features:  10 

Accuracy using ALL features: 94.77
Accuracy using Chi2: 93.35
Accuracy using Mutual Information: 91.75


Since feature reduction always decreases accuracy comapred to using all features, we have decided to only train further models using all features.

### 3. Compute accuracy of Gaussian Naive Bayes

In [10]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report

model = GaussianNB()
model.fit(X_train_enc, y_train_enc)
pred = model.predict(X_test_enc)

# 3. evaluate model
accuracy = accuracy_score(y_test_enc, pred)
print('Accuracy of Naive Bayes: %.2f' % (accuracy*100))

# 3b. confusion matrix
cm = confusion_matrix(y_test_enc, pred)  # actual, pred
print("\nConfusion matrix raw: ")
print(cm)

print("\nClassification Report : ")
print(classification_report(y_test_enc, pred))

Accuracy of Naive Bayes: 91.63

Confusion matrix raw: 
[[753  67]
 [ 69 736]]

Classification Report : 
              precision    recall  f1-score   support

           0       0.92      0.92      0.92       820
           1       0.92      0.91      0.92       805

    accuracy                           0.92      1625
   macro avg       0.92      0.92      0.92      1625
weighted avg       0.92      0.92      0.92      1625



### 4. Compute accuracy of Neural Network using All Features

#### 4a. Hyperparameter tuning

In [ ]:
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.model_selection import GridSearchCV

# Performing some hyperparameter tuning (it can take aorund 3 minutes)
max_iterations = [200,500,800,1000]
hidden_layer_siz = [(100,), (50,), (100, 25), (25, 7), (70, 20), (12, 10), (5, 20)]
learning_rates = 0.1 * np.arange(1, 4)
param_grid = dict(learning_rate_init = learning_rates, hidden_layer_sizes = hidden_layer_siz, max_iter = max_iterations)

#set model
mlp = MLPClassifier(solver = 'adam', random_state = 42, activation = 'logistic', learning_rate_init = 0.3, batch_size = 100, hidden_layer_sizes = (12, 3), max_iter = 500)

# For Grid Search
grid = GridSearchCV(estimator = mlp, param_grid = param_grid)

# Train the model with grid search
grid_result = grid.fit(X_train_enc, y_train_enc)

# Print the best hyperparameters
print("Best parameters: ", grid.best_params_)



Best parameters:  {'hidden_layer_sizes': (100,), 'learning_rate_init': 0.1, 'max_iter': 200}


#### 4b. Fitting the model with the tuned hyperparameters

In [ ]:
from sklearn.metrics import classification_report, multilabel_confusion_matrix

#set model with the best hyperparameters that we just computed
mlp = MLPClassifier(solver = 'adam', random_state = 42, activation = 'logistic', learning_rate_init = grid_result.best_params_["learning_rate_init"], batch_size = 100, hidden_layer_sizes = grid_result.best_params_["hidden_layer_sizes"], max_iter = grid_result.best_params_["max_iter"])

mlp.fit(X_train_enc, y_train_enc)
pred = mlp.predict(X_test_enc)

accuracy = accuracy_score(y_test_enc, pred)
print('Accuracy: %.2f' % (accuracy*100))

print("\nConfusion Matrix for each label : ")
print(multilabel_confusion_matrix(y_test_enc, pred))

print("\nClassification Report : ")
print(classification_report(y_test_enc, pred))


Accuracy: 100.00

Confusion Matrix for each label : 
[[[805   0]
  [  0 820]]

 [[820   0]
  [  0 805]]]

Classification Report : 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       820
           1       1.00      1.00      1.00       805

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625



#### 4c. k-fold cross validation

In [ ]:
# Using sklearn function cross_validate()

from sklearn.model_selection import cross_validate

CV = cross_validate(mlp, X_test_enc, y_test_enc, cv=10, scoring=['accuracy', 'neg_mean_squared_error'])
print('Accuracy')
print(CV['test_accuracy'])
print('MSE')
print(-1*CV['test_neg_mean_squared_error'])

print('Average Accuracy = ', sum(CV['test_accuracy']) / len(CV['test_accuracy']))
print('Average MSE = ', sum(-1 * CV['test_neg_mean_squared_error']) / len(CV['test_neg_mean_squared_error']))

Accuracy
[1.         0.99386503 1.         0.98159509 1.         1.
 1.         1.         1.         1.        ]
MSE
[0.         0.00613497 0.         0.01840491 0.         0.
 0.         0.         0.         0.        ]
Average Accuracy =  0.9975460122699387
Average MSE =  0.00245398773006135


In [ ]:
# To find list of accuracy and MSE values
# Without using the sklearn function cross_validate()

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# Pre-process data for K-Folding
df = read_csv('mushrooms.csv')
X = df.drop('class', axis = 1)
y = df['class']

oe = OrdinalEncoder()
oe.fit(X)
X = oe.transform(X)

le = LabelEncoder()
le.fit(y)
y = le.transform(y)

# step 1: randomize the dataset and create k equal size partitions
n_splits=10
kf = KFold(n_splits=n_splits)

acc = 0
mse = 0

i = 0 #keep track of batch number

# step 5: iterate k times with a different testing subset
for train_indices, test_indices in kf.split(X):

    # step 2-3: use k-1/k^th partition for the training/testing model
    start_train, stop_train = train_indices[0], train_indices[-1]+1
    start_test, stop_test = test_indices[0], test_indices[-1]+1
    
    # perform the training similar to Q1
    #this was based on the requirements in Q1
    mlp = MLPClassifier(solver = 'sgd', random_state = 42, activation = 'logistic', learning_rate_init = 0.3, batch_size = 100, hidden_layer_sizes = (12, 3), max_iter = 500)
    mlp.fit(X[start_train:stop_train], y[start_train:stop_train])
    pred = mlp.predict(X[start_test:stop_test])
    
    # step 4: record the evaluating scores
    i+=1
    acc += accuracy_score(y[start_test:stop_test], pred)
    mse += mean_squared_error(y[start_test:stop_test], pred)
    
    print("\nAccuracy for batch ", i, " : ", accuracy_score(y[start_test:stop_test], pred))
    print("Mean Square Error for batch ", i, " : ", mean_squared_error(y[start_test:stop_test], pred))

# step 6: find the average and select the batch with highest evaluation scores
print('\nAverage Accuracy = ', acc / n_splits)
print('Average MSE = ', mse / n_splits)


Accuracy for batch  1  :  1.0
Mean Square Error for batch  1  :  0.0

Accuracy for batch  2  :  1.0
Mean Square Error for batch  2  :  0.0

Accuracy for batch  3  :  1.0
Mean Square Error for batch  3  :  0.0

Accuracy for batch  4  :  1.0
Mean Square Error for batch  4  :  0.0

Accuracy for batch  5  :  1.0
Mean Square Error for batch  5  :  0.0

Accuracy for batch  6  :  0.9987684729064039
Mean Square Error for batch  6  :  0.0012315270935960591

Accuracy for batch  7  :  0.9926108374384236
Mean Square Error for batch  7  :  0.007389162561576354

Accuracy for batch  8  :  0.9987684729064039
Mean Square Error for batch  8  :  0.0012315270935960591

Accuracy for batch  9  :  1.0
Mean Square Error for batch  9  :  0.0

Accuracy for batch  10  :  0.979064039408867
Mean Square Error for batch  10  :  0.020935960591133004

Average Accuracy =  0.9969211822660098
Average MSE =  0.0030788177339901475
